In [3]:
import os
current_dir = os.getcwd()
csv_dir = os.path.join(current_dir,'每日各站進出站人數')
files_and_dirs = os.listdir(csv_dir)
files_and_dirs

['每日各站進出站人數2022.csv',
 '每日各站進出站人數2023.csv',
 '每日各站進出站人數20190423-20191231.csv',
 '每日各站進出站人數2021.csv',
 'manifest.csv',
 '每日各站進出站人數2020.csv',
 'schema.csv']

In [5]:
for filename in files_and_dirs:
    if '每日各站進出站人數' in filename:
        print(filename)

每日各站進出站人數2022.csv
每日各站進出站人數2023.csv
每日各站進出站人數20190423-20191231.csv
每日各站進出站人數2021.csv
每日各站進出站人數2020.csv


In [8]:
all_csv_files = [filename for filename in files_and_dirs if '每日各站進出站人數' in filename]

In [10]:
sorted_csv_files = sorted(all_csv_files)

In [15]:
def parse_data(filename):
    print(filename)

for filename in sorted_csv_files:
    abs_filename = os.path.join(csv_dir,filename)
    parse_data(abs_filename)
    

/workspaces/__2024_07_29_1_3_night__/lesson19/每日各站進出站人數/每日各站進出站人數20190423-20191231.csv
/workspaces/__2024_07_29_1_3_night__/lesson19/每日各站進出站人數/每日各站進出站人數2020.csv
/workspaces/__2024_07_29_1_3_night__/lesson19/每日各站進出站人數/每日各站進出站人數2021.csv
/workspaces/__2024_07_29_1_3_night__/lesson19/每日各站進出站人數/每日各站進出站人數2022.csv
/workspaces/__2024_07_29_1_3_night__/lesson19/每日各站進出站人數/每日各站進出站人數2023.csv
